In [1]:
## this workflow follows the tutorial published by Qiusheng Wu, Assistant Professor of Geography at the University of Tennessee, Knoxville

## https://github.com/giswqs/geemap
## https://www.youtube.com/watch?v=k9MEy2awVJQ&list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3&index=33&ab_channel=QiushengWuQiushengWu

In [ ]:
## imports Earth Engine library
import os
import ee

In [2]:
## imports geemap package

import geemap

In [3]:
## creates interactive map

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
## defines the point of interest on the map
point = ee.Geometry.Point([-59.3, 47.833333])

## filters through Landsat image collection using the input parameters
## landsat 8 tier 1 surface reflectance data
## filters the boundaries of the data by the defined point
## selects only images from the selected dates
## sorts by cloud cover
## selects the first image with the least cloud cover
## selects bands 1-7

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')                                      


## image symbology
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']                            ## bands 5, 4, 3
}

Map.centerObject(point, 8)                                 ## centers the map around the defined point
Map.addLayer(image, vis_params, "Landsat-8")               ## adds image layer to the map

In [5]:
## populates image metadata

props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.07,
 'CLOUD_COVER_LAND': 0.23,
 'EARTH_SUN_DISTANCE': 0.998913,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 6.004,
 'GEOMETRIC_RMSE_MODEL_X': 4.353,
 'GEOMETRIC_RMSE_MODEL_Y': 4.135,
 'IMAGE_DATE': '2019-10-09',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_005027_20191009_20191018_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1571411851000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-10-09T14:42:38.3008550Z',
 'SOLAR_AZIMUTH_ANGLE': 162.203156,
 'SOLAR_ZENITH_ANGLE': 55.285503,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 5,
 'WRS_ROW': 27,
 'system:asset_size': '376.186106 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_005027_20191009',
 'system:index': 'LC08_005027_20191009',
 'system:time_end': '2019-10-09 14:42:38',
 'system:time_start': '2019-10-09 14:42:38',
 'system:version': 15717396395

In [6]:
## get image date (also in metadata)

props.get('IMAGE_DATE').getInfo()

'2019-10-09'

In [7]:
## get cloud cover info (also in metadata)

props.get('CLOUD_COVER').getInfo()

0.07

In [8]:
## draw a polygon on the map to use for the training dataset
## this is not necessary, and can be left to default. will take samples from the entire image if left blank

region = Map.user_roi

In [9]:
## make training dataset

training = image.sample(**{                         ## randomly samples points to be used for training data 
    'region': region,                               ## user can define a specific region for samples, it will default if not
    'scale': 30,                                    ## spatial resolution
    'numPixels': 5000,                              ## number of points selected for training
    'seed': 0,                                      ## seed is random, selecting 0 will set the number to be the same each time
    'geometries': True                              ## Set this to False to ignore points on the map
})

Map.addLayer(training, {}, 'training', False)       ## adds training sample layer to the map
Map

Map(bottom=23127.0, center=[47.83333300000001, -59.300000000000004], controls=(WidgetControl(options=['positio…

In [10]:
## train the clusterer

n_clusters = 5                                                       ## specifies the number of clusters (0-4)
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)      ## using Weka KMeans method

In [11]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=23127.0, center=[47.83333300000001, -59.300000000000004], controls=(WidgetControl(options=['positio…

In [12]:
## label the clusters

legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']          ## keep it general for the first run, rename after running this section
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

## Reclassify the map
## goes from labeled as 0-4 to 1-5
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5]) ## COMMENT OUT THIS SECTION WHEN RE-RUNNING OR ELSE IT WILL RECLASSIFY THE MAP

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')     ## adds newly classified layer to map
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')  ## adds legend to map
Map

Map(bottom=23127.0, center=[47.83333300000001, -59.300000000000004], controls=(WidgetControl(options=['positio…

In [13]:
## creates a slider to visualize and compare the results

print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [14]:
## export results to computer

import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\ashle\Downloads\cluster.tif


In [15]:
## sets download directory for HTML file export

download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'my_map.html')

In [16]:
## save as HTML file

Map.to_html(outfile=html_file, title='Unsupervised Classification', width='100%', height='880px')